In [ ]:
!pip install torch torchdata
!pip install transformers datasets
!pip install -q -U bitsandbytes
!pip install -U accelerate
!pip install evaluate rouge_score
!pip install -qqq trl
!pip install loralib peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is i

In [ ]:
pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.34
    Uninstalling SQLAlchemy-2.0.34:
      Successfully uninstalled SQLAlchemy-2.0.34
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer,DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
from datasets import load_dataset, DatasetDict
import accelerate

import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from transformers import DataCollatorForSeq2Seq


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

# Example: Min-Max Scaling
def min_max_normalize(values):
    scaler = MinMaxScaler()
    return scaler.fit_transform(np.array(values).reshape(-1, 1)).flatten()

def min_max_denormalize(normalized_values, scaler):
    return scaler.inverse_transform(np.array(normalized_values).reshape(-1, 1)).flatten()

# Example: Standardization (Z-score normalization)
def standardize(values):
    scaler = StandardScaler()
    return scaler.fit_transform(np.array(values).reshape(-1, 1)).flatten()

def destandardize(standardized_values, scaler):
    return scaler.inverse_transform(np.array(standardized_values).reshape(-1, 1)).flatten()


In [ ]:
seperator_print_short = '-'.join('' for x in range(100))
seperator_print_long = '-'.join('' for x in range(140))

def print_input_outputs(i, index, input, label, model_output):
    print(seperator_print_long)

    print(f"Sample {i+1}: Index = {index}")
    print(seperator_print_short)

    print(f"INPUT:")
    print(input)
    print(seperator_print_short)

    print("DATASET ORIGINAL LABEL:")
    print(label)
    print(seperator_print_short)

    if model_output != "":
        print("OUTPUT OF THE MODEL")
        print(model_output)
        print(seperator_print_short)

    print(seperator_print_long)



In [ ]:
dataset_src = "/content/dataset.csv"
dataset_origin = load_dataset('csv',
                       data_files=dataset_src
                       )
dataset_origin

DatasetDict({
    train: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 500607
    })
})

In [ ]:
dset_train_test = (load_dataset('csv',
                        data_files=dataset_src,
                        split='train'
                        )
          .train_test_split(train_size=0.7)
          )

dset_train_test

DatasetDict({
    train: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 350424
    })
    test: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 150183
    })
})

In [ ]:
dset_test_val = dset_train_test["test"].train_test_split(train_size=0.75)

dset_test_val

DatasetDict({
    train: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 112637
    })
    test: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 37546
    })
})

In [ ]:
dataset = DatasetDict({
                      'train': dset_train_test['train'],
                      'validation': dset_test_val['train'],
                      'test': dset_test_val['test']}
                      )

dataset

DatasetDict({
    train: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 350424
    })
    validation: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 112637
    })
    test: Dataset({
        features: ['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'],
        num_rows: 37546
    })
})

In [ ]:
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])


In [ ]:
train_df

,Ligand SMILES,Ligand InChI,Target Name,Ki (nM)
0,CC[C@H](C)[C@H](N(C)C(C)=O)C(=O)N[C@H]1CCc2ccc...,InChI=1S/C28H36N4O9/c1-6-14(2)22(31(4)15(3)33)...,Granzyme B,16.00
1,CC(C)[C@H](NC(=O)c1cccnc1)C(=O)N1Cc2ccccc2C[C@...,InChI=1S/C32H40N6O5/c1-18(2)24(26(39)29-36-37-...,Neutrophil elastase,1.66
2,C[C@]1(CS(=O)(=O)[C@@](C)(C2CCOCC2)C(=N)N1)c1c...,InChI=1S/C17H24FN3O3S/c1-16(13-9-12(19)3-4-14(...,Beta-secretase 1,500.00
3,CCCOC(=O)c1cnc2n(CC(Cl)c3ccccc3)ncc2c1N1CCCC1,InChI=1S/C22H25ClN4O2/c1-2-12-29-22(28)18-13-2...,Adenosine receptor A2a,1988.00
4,COc1cc(C=Nc2c(C#N)c(cn2-c2ccc(cc2)S(N)(=O)=O)-...,InChI=1S/C25H19ClN4O4S/c1-34-24-12-16(2-11-23(...,Carbonic anhydrase 9,785.00
...,...,...,...,...
350419,O=c1n([se]c2ccccc12)-c1ccccc1,InChI=1S/C13H9NOSe/c15-13-11-8-4-5-9-12(11)16-...,"Indoleamine 2,3-dioxygenase 1",94000.00
350420,CC(C)N1CCC[C@H](Cn2c(nc3ccc(Oc4ccc(F)cc4)cc3c2...,InChI=1S/C26H32FN3O2/c1-17(2)25-28-24-12-11-22...,Growth hormone secretagogue receptor type 1,4.00
350421,CN(C)S(=O)(=O)N[C@H]1C[C@H]2[C@@H](Cc3cn(C)c4c...,"InChI=1S/C18H26N4O2S/c1-20(2)25(23,24)19-13-9-...",5-hydroxytryptamine receptor 2B,2.00
350422,CC(=O)NCCC1CCCc2cc3OCCOc3cc12,InChI=1S/C16H21NO3/c1-11(18)17-6-5-12-3-2-4-13...,Melatonin receptor type 1A/1B,62.00


In [ ]:
train_df["Ki (nM)"].describe()

,Ki (nM)
count,3.504240e+05
mean,3.229441e+05
std,1.155266e+07
min,0.000000e+00
25%,7.300000e+00
50%,7.200000e+01
75%,8.200000e+02
max,3.000000e+09


In [ ]:
val_df["Ki (nM)"].describe()

,Ki (nM)
count,1.126370e+05
mean,1.775929e+09
std,4.213787e+11
min,0.000000e+00
25%,7.200000e+00
50%,7.200000e+01
75%,8.260000e+02
max,1.000000e+14


In [ ]:
test_df["Ki (nM)"].describe()

,Ki (nM)
count,3.754600e+04
mean,2.790440e+05
std,1.060160e+07
min,0.000000e+00
25%,7.300000e+00
50%,7.600000e+01
75%,8.627500e+02
max,9.908319e+08


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datasets import Dataset


# Convert datasets to pandas DataFrames
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on training data and transform all datasets
train_df['Ki (nM)_normalized'] = scaler.fit_transform(train_df[['Ki (nM)']])
val_df['Ki (nM)_normalized'] = scaler.transform(val_df[['Ki (nM)']])
test_df['Ki (nM)_normalized'] = scaler.transform(test_df[['Ki (nM)']])

# Update datasets
from datasets import Dataset

# Assuming `train_df`, `val_df`, and `test_df` are your pandas DataFrames
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
})

print(dataset)


KeyboardInterrupt: 

In [ ]:
train_df["Ki (nM)_normalized"].describe()


,Ki (nM)_normalized
count,3.504240e+05
mean,2.856763e-06
std,1.689286e-03
min,0.000000e+00
25%,7.300000e-14
50%,7.300000e-13
75%,8.200000e-12
max,1.000000e+00


In [ ]:
val_df["Ki (nM)_normalized"].describe()


,Ki (nM)_normalized
count,1.126370e+05
mean,8.881407e-06
std,2.979610e-03
min,0.000000e+00
25%,7.100000e-14
50%,7.200000e-13
75%,8.220000e-12
max,1.000000e+00


In [ ]:
test_df["Ki (nM)_normalized"].describe()

,Ki (nM)_normalized
count,3.754600e+04
mean,3.634356e-09
std,1.285111e-07
min,0.000000e+00
25%,7.300000e-14
50%,7.400000e-13
75%,8.490000e-12
max,1.000000e-05


In [ ]:
 from transformers import BitsAndBytesConfig

 BNB_NF4_CONFIG = BitsAndBytesConfig(
     load_in_4bit=True,
     bnb_4bit_use_double_quant=True,
     bnb_4bit_quant_type="nf4",
     bnb_4bit_compute_dtype=torch.bfloat16
 )

In [ ]:
model_id='seyonec/ChemBERTa-zinc-base-v1'

model = AutoModelForSequenceClassification.from_pretrained(model_id,
                                              quantization_config = BNB_NF4_CONFIG,
                                              device_map = "cuda",
                                              num_labels=1
                                              )

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_fast=True,
                                          )




Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
INPUT_COL_NAME1 = "Ligand SMILES"
INPUT_COL_NAME2 = "Target Name"
OUTPUT_COL_NAME = "Ki (nM)"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import default_data_collator
from peft import LoraConfig, get_peft_model



# LoRA configuration for ChemBERTa
LORA_R = 8
LORA_ALPHA = 64
LORA_DROPOUT = 0.01

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["query", "value"],  # Adjust based on ChemBERTa
    lora_dropout=LORA_DROPOUT,
    bias="lora_only",
)

# Apply LoRA to the ChemBERTa model
peft_model = get_peft_model(model, lora_config)


In [ ]:

###def tokenize_function(examples):
    INPUT_COL_NAME1 = "Ligand SMILES"
    INPUT_COL_NAME2 = "Target Name"
    OUTPUT_COL_NAME = "Ki (nM)"  # Updated to normalized column

    START_prompt = 'Predict the Ki (nM) value for the following drug-target interaction:'
    SMILES_prompt = [f'\nLigand SMILES: {smiles}' for smiles in examples[INPUT_COL_NAME1]]
    TARGET_prompt = [f'\nTarget Name: {target}' for target in examples[INPUT_COL_NAME2]]
    END_prompt = '\nPredicted Ki (nM): '

    # Combine the input prompts for each example
    prompts = [START_prompt + s + t + END_prompt for s, t in zip(SMILES_prompt, TARGET_prompt)]

    # Tokenize the input prompts and pad/truncate
    inputs = tokenizer(prompts, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

    # Tokenize the output (Ki (nM)_normalized) values and pad/truncate
    labels = tokenizer([str(ki) for ki in examples[OUTPUT_COL_NAME]],
                       truncation=True, padding='max_length', max_length=128)

    # Return tokenized inputs and labels
    return {
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze(),
        'labels': labels['input_ids']
    }


# Apply the tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns from the dataset
tokenized_datasets = tokenized_datasets.remove_columns(['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)'])

# Define the data collator for sequence classification
data_collator = default_data_collator


In [ ]:
def tokenize_function(examples):
    INPUT_COL_NAME1 = "Ligand SMILES"
    INPUT_COL_NAME2 = "Target Name"
    OUTPUT_COL_NAME = "Ki (nM)_normalized"  # Ensure this column exists in the dataset

    START_prompt = 'Predict the Ki (nM) value for the following drug-target interaction:'
    SMILES_prompt = [f'\nLigand SMILES: {smiles}' for smiles in examples[INPUT_COL_NAME1]]
    TARGET_prompt = [f'\nTarget Name: {target}' for target in examples[INPUT_COL_NAME2]]
    END_prompt = '\nPredicted Ki (nM): '

    # Combine the input prompts for each example
    prompts = [START_prompt + s + t + END_prompt for s, t in zip(SMILES_prompt, TARGET_prompt)]

    # Tokenize the input prompts and pad/truncate
    inputs = tokenizer(prompts, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

    # Convert the output (Ki (nM)_normalized) values to a tensor of floats
    labels = [float(ki) for ki in examples[OUTPUT_COL_NAME]]
    labels_tensor = torch.tensor(labels, dtype=torch.float)

    # Return tokenized inputs and labels
    return {
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze(),
        'labels': labels_tensor
    }


    # Apply the tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns from the dataset
tokenized_datasets = tokenized_datasets.remove_columns(['Ligand SMILES', 'Ligand InChI', 'Target Name', 'Ki (nM)',"Ki (nM)_normalized"])

# Define the data collator for sequence classification
data_collator = default_data_collator

Map:   0%|          | 0/350424 [00:00<?, ? examples/s]

Map:   0%|          | 0/112637 [00:00<?, ? examples/s]

Map:   0%|          | 0/37546 [00:00<?, ? examples/s]

In [ ]:

# Define the training arguments
peft_training_args = TrainingArguments(
    output_dir='output_dir',
    num_train_epochs=2,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    optim='paged_adamw_32bit',
    do_train=True,
    do_eval=True,
    evaluation_strategy='epoch',
    logging_steps=4,
    metric_for_best_model='eval_loss',

)

# Initialize the Trainer
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Start training
peft_trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:


def tokenize_test_function(examples):
    INPUT_COL_NAME1 = "Ligand SMILES"
    INPUT_COL_NAME2 = "Target Name"

    START_prompt = 'Predict the Ki (nM) value for the following drug-target interaction:'
    SMILES_prompt = [f'\nLigand SMILES: {smiles}' for smiles in examples[INPUT_COL_NAME1]]
    TARGET_prompt = [f'\nTarget Name: {target}' for target in examples[INPUT_COL_NAME2]]
    END_prompt = '\nPredicted Ki (nM): '

    # Combine the input prompts for each example
    prompts = [START_prompt + s + t + END_prompt for s, t in zip(SMILES_prompt, TARGET_prompt)]

    # Tokenize the input prompts and pad/truncate
    inputs = tokenizer(prompts, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

    # Return tokenized inputs without labels
    return {
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze()
    }

# Apply the tokenization to the test dataset
tokenized_test_dataset = dataset["test"].map(tokenize_test_function, batched=True)


Map:   0%|          | 0/37546 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test dataset
test_results = peft_trainer.evaluate(eval_dataset=tokenized_test_dataset)

# Print evaluation results
print(test_results)


In [ ]:
import torch

def predict_on_test_dataset(trainer, test_dataset):
    # Generate predictions
    predictions = trainer.predict(test_dataset).predictions
    predictions = torch.tensor(predictions).numpy()

    # Process the predictions if needed
    # For example, you might want to convert them to actual values or ranks
    # predictions = process_predictions(predictions)

    return predictions

# Make predictions on the test dataset
predictions = predict_on_test_dataset(peft_trainer, tokenized_test_dataset)

# Print or save predictions
print(predictions)
